In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/flight-take-off-data-jfk-airport/M1_final.csv")
df.head()
# df.shape

In [ ]:
df.isnull().sum()
df.dropna(axis = 0,inplace = True)
df.drop('TAIL_NUM',axis=1,inplace=True)
df.shape

In [ ]:
X = df.drop(['TAXI_OUT'],axis = 1)
X.head()
y = df.TAXI_OUT
X.shape
y.shape

In [ ]:
df.columns

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X_train.Condition = labelencoder.fit_transform(X_train.Condition)
X_train.head(20)

# Label Encoding

In [ ]:
# df.dtypes
X_train.OP_UNIQUE_CARRIER = labelencoder.fit_transform(X_train.OP_UNIQUE_CARRIER)
#X_train.TAIL_NUM = labelencoder.fit_transform(X_train.TAIL_NUM)
X_train.DEST = labelencoder.fit_transform(X_train.DEST)
X_train['Dew Point'] = labelencoder.fit_transform(X_train['Dew Point'])
X_train['Wind'] = labelencoder.fit_transform(X_train['Wind'])

# Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
X_train = standardscaler.fit_transform(X_train)
X_train.shape

# Linear Regression - Label Encoding

In [ ]:
from sklearn.linear_model import LinearRegression
linear = LinearRegression()
linear.fit(X_train,y_train)
X_test.Condition = labelencoder.fit_transform(X_test.Condition)
X_test.OP_UNIQUE_CARRIER = labelencoder.fit_transform(X_test.OP_UNIQUE_CARRIER)
#X_test.TAIL_NUM = labelencoder.fit_transform(X_test.TAIL_NUM)
X_test.DEST = labelencoder.fit_transform(X_test.DEST)
X_test['Dew Point'] = labelencoder.fit_transform(X_test['Dew Point'])
X_test['Wind'] = labelencoder.fit_transform(X_test['Wind'])
X_test = standardscaler.fit_transform(X_test)
y_actual = linear.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
linError = mean_squared_error(y_test,y_actual,squared = False)
linError
# error_linear_regression    

# Ridge Regression - Label Encoding

In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge(alpha = 1.0)
ridge.fit(X_train,y_train)
y_pred=ridge.predict(X_test)
ridgeError=mean_squared_error(y_test,y_pred,squared = False)
ridgeError

# Lasso Regression - Label Encoding

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha = 0.01)
lasso.fit(X_train,y_train)
y_pd=lasso.predict(X_test)
lassoError=mean_squared_error(y_test,y_pd,squared = False)
lassoError

# K Nearest Neighbours - Label Encoding

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors = 11)
knn.fit(X_train,y_train)
ypd = knn.predict(X_test)
knnError = mean_squared_error(y_test,ypd,squared = False)
knnError

# SVR - Label Encoding

In [ ]:
from sklearn.svm import SVR
svr = SVR(C=20)
svr.fit(X_train,y_train)
yp = svr.predict(X_test)
svrError = mean_squared_error(y_test,yp,squared = False)
svrError


# Random Forests - Label Encoding

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=100)
rfr.fit(X_train,y_train)
yd = rfr.predict(X_test)
rfrError = mean_squared_error(y_test,yd,squared = False)
rfrError

# Light GBM - Label Encoding

In [ ]:
from lightgbm import LGBMRegressor
lgbm=LGBMRegressor()
lgbm.fit(X_train,y_train)
yl=lgbm.predict(X_test)
lgbError=mean_squared_error(y_test,yl,squared=False)
lgbError

# Naive Bayes - Label Encoding

In [ ]:
from sklearn.linear_model import BayesianRidge
bayreg=BayesianRidge()
bayreg.fit(X_train,y_train)
yb=bayreg.predict(X_test)
bayError=mean_squared_error(y_test,yb,squared=False)
bayError


# One hot encoding

In [ ]:
cols = ['OP_UNIQUE_CARRIER','DEST','Dew Point','Wind','Condition']

In [ ]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_df = pd.DataFrame(OH_encoder.fit_transform(df[cols]))

In [ ]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols_df = pd.DataFrame(OH_encoder.fit_transform(df[cols]))
OH_cols_df.index = df.index

# Remove categorical columns (will replace with one-hot encoding)
num_df = df.drop(cols, axis=1)


# Add one-hot encoded columns to numerical features
OH_df = pd.concat([num_df, OH_cols_df], axis=1)

OH_df.head()
y=OH_df.TAXI_OUT
X=OH_df.drop(['TAXI_OUT'], axis =1)

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state = 50)

In [ ]:
X_train

# Linear Regression - One hot encoding

In [ ]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_train,y_train)
ylr = lr.predict(X_test)
from sklearn.metrics import mean_squared_error
elr = mean_squared_error(ylr,y_test,squared = False)
elr

# Lasso Regression - one Hot Encoding

In [ ]:
from sklearn.linear_model import Lasso
lsr = Lasso(alpha=0.001)
lsr.fit(X_train,y_train)
ylsr = lsr.predict(X_test)
from sklearn.metrics import mean_squared_error
elsr = mean_squared_error(ylsr,y_test,squared = False)
elsr

# Ridge Regression - one Hot Encoding

In [ ]:
from sklearn.linear_model import Ridge
rsr = Ridge(alpha=50)
rsr.fit(X_train,y_train)
yrsr = rsr.predict(X_test)
from sklearn.metrics import mean_squared_error
ersr = mean_squared_error(yrsr,y_test,squared = False)
ersr

# K Nearest Neighbours - One Hot encoding

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors = 12)
knn.fit(X_train,y_train)
yknn = knn.predict(X_test)
eknn = mean_squared_error(y_test,yknn,squared = False)
eknn

# SVR - One Hot Encoding

In [ ]:
from sklearn.svm import SVR
svr = SVR()
svr.fit(X_train,y_train)
ysvr = svr.predict(X_test)
esvr = mean_squared_error(y_test,ysvr,squared = False)
esvr

# Random Forest - One Hot Encoding

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(n_estimators=500)
rfr.fit(X_train,y_train)
yrfr = rfr.predict(X_test)
erfr = mean_squared_error(y_test,yrfr,squared = False)
erfr

# Light GBM  - One Hot encoding

In [ ]:
from lightgbm import LGBMRegressor
lgbm=LGBMRegressor()
lgbm.fit(X_train,y_train)
ylgm=lgbm.predict(X_test)
elgb=mean_squared_error(y_test,yl,squared=False)
elgb

# Naive Bayes - One Hot Encoding

In [ ]:
from sklearn.linear_model import BayesianRidge
bayreg=BayesianRidge()
bayreg.fit(X_train,y_train)
ybay=bayreg.predict(X_test)
ebay=mean_squared_error(y_test,yb,squared=False)
ebay

In [ ]:
label_error = [linError,ridgeError,lassoError,knnError,svrError,rfrError,bayError,lgbError]
ohe_error = [elr,elsr,ersr,eknn,esvr,erfr,ebay,elgb]

In [ ]:
plt.plot(["linear","Lasso","Ridge","KNN","SVR","RF","NB","LGBM"],label_error,"g",label="label encoding",linewidth=1)
plt.plot(["linear","Lasso","Ridge","KNN","SVR","RF","NB","LGBM"],ohe_error,"b",label="one hot encoding",linewidth=1)
plt.title("error analysis")
plt.xlabel("different model")
plt.ylabel("root mean squared error")
plt.legend() 
plt.show

In [ ]:
#Therefore, random forest which works on the principle of using various decision trees being used to evaluate various models simultaneouly 
#claims the most accurate results relative to others

# 